In [1]:
###https://stackoverflow.com/questions/52889704/python-win32com-excel-com-model-started-generating-errors
### print(win32com.__gen_path__) #to find the win32com dir and delete it if there is error about 
### or try this: python -m win32com.client.makepy "Excel.Application"

C:\Users\bt\AppData\Local\Temp\gen_py\3.8


In [1]:
# clean up verion 4/17
### puting together the solution 
import win32com.client as win32
from win32com.client import constants as C
import datetime 
import time
import logging 
from logging import debug as lgd
from logging import info as lgi
from logging import error as lge
from dateutil import tz 
from a_Stock_IF import Stock
import a_utils
import a_TDA_IF


In [2]:
# logging for debugging 
#
#yfilter=a_utils.LevelFilter((logging.CRITICAL, logging.WARNING,  logging.INFO, logging.ERROR, logging.DEBUG))
yfilter=a_utils.LevelFilter((logging.INFO, logging.CRITICAL))
lg=a_utils.BTLogger(stream_filter=yfilter)


+__init__\217|INFO: logging started at:04/24/21 [a_utils.py 04/24:09:56]


In [7]:
def function1a():
    a=1
    lgd (' checking function from lgd')

In [6]:
# testing a_OL_IF.yPY
import a_OL_IF 

yWD= win32.gencache.EnsureDispatch("Word.Application")  # gencache.EnsureDispatch for wdConstant enumeration
yOL = win32.gencache.EnsureDispatch("Outlook.Application")  #w, needed for importing constants:
yNS = yOL.GetNamespace("MAPI")
yFolder = yNS.Folders['BXSelfCurrent'].Folders['BTHM'].Folders['0-outlook usage'].Folders['Test Run Outlook Usage'].Folders['Securities']

lgd ('checking import of logging.debug directly')
for yOLI in yFolder.Items:

    lg.log_stream.seek(0)
    lg.log_stream.truncate(0)
    
    print('--------------------')
    yStock=a_OL_IF.InitStock (yOLI)
     


    yStock.GetHist() # include TA and saving 
    
    print('----')
    
   
    a_OL_IF.UpdateOLI1(yOLI, yStock, lg)
    print('--')

    print ('>>>>OL comment:'+lg.log_stream.getvalue())
    
    
    
#yWD.Quit() 
#yOL.Quit() #w, but you want outlook to be on//




+<module>\9|DEBUG: checking import of logging.debug directly [<ipython-input-6-790802539f62> 04/24:12:19]
--------------------


AttributeError: module 'a_OL_IF' has no attribute 'InitStock'

In [2]:
def InitStock (yOLI):

    yAlert=''
    try:
        ySymbol= yOLI.UserProperties.Find("Sec").Value
    except:
        ySymbol = "QQQ"
        yAlert=r'Alert: can\'t get symbol from OLI Sec field; use QQQ instead' 
        
    yStock=Stock(ySymbol)
    yStock.Comment=yStock.Comment+r'\r\n InitStock():'+ yAlert + '; '

    yStock.HistEndDate=a_utils.epoch_date_stamp()
    yStock.HistStartDate=a_utils.epoch_from_today( Yr=1, Mo=0, Day=0) 

    try:
        yStock.SMAState=yOLI.UserProperties.Find("SMAState").Value
    except: 
        yStock.SMAState = 0
        Stock.Comment=yStock.Comment+'SMAState initialized to 0 '  + '; '

    try:
        yStock.SMAAlert=yOLI.UserProperties.Find("SMAAlert").Value
    except: 
        yStock.SMAAlert = 0
        Stock.Comment=yStock.Comment+'SMAAert initialized to 0 '  + '; '
    try:
        yStock.SMADays=yOLI.UserProperties.Find("SMADays").Value
    except: 
        yStock.SMADays = 10
        Stock.Comment=yStock.Comment+'SMADays initialized to 10 '  + '; '


    print(yStock.Comment)
    return yStock

In [3]:
def SetOLIUsrProp(OLItem, Fieldnm, Value, FieldType=1):
#find field type from here
# https://docs.microsoft.com/en-us/office/vba/api/outlook.oluserpropertytype
# C.olCurrency=14, C.olDateTime=5, C.olText=1, C.olNumber=3 

    print('SetOLIUsrProp=', Fieldnm, ',', Value, ',',FieldType)
    
    if OLItem.UserProperties.Find(Fieldnm) == None:
        try:
            yOLI.UserProperties.Add(Fieldnm, FieldType)
            yOLI.UserProperties.Find(Fieldnm).Value=Value
            print ('SetOLIUserProp() ', Fieldnm, ',' , Value,', added')
        except:
            print ('SetOLIUserProp() ', Fieldnm, ',' , Value,' add failed')
    else:
        #nw OLItem.UserProperties.Find(Fieldnm).Value=Value
        try: 
            yPA= OLItem.PropertyAccessor
            yProp=r"http://schemas.microsoft.com/mapi/string/{00020329-0000-0000-C000-000000000046}/" + Fieldnm
            yPA.SetProperty(yProp,Value)

            print ('SetOLIUserProp() ', Fieldnm, ',' , Value,', accessor done')
        except:             
            print ('SetOLIUserProp() ', Fieldnm, ',' , Value,'  accessor failed ')
    return


In [4]:
def UpdateOLIFields(yOLI, yStock):
    try: 
        #yStock.Comment=yStock.Comment+' UpdateOLIFields() ??? '  + '; '
  
        ## ySymbol =yOLI.UserProperties.Find("Sec").Value
     
        #yOLI.UserProperties.Find("SMAState").Value=yStock.SMAState
        SetOLIUsrProp(yOLI, "SMAState", yStock.SMAState , C.olNumber)
            
        #yOLI.UserProperties.Find("SMAAlert").Value=yStock.SMAAlert
        SetOLIUsrProp(yOLI, "SMAAlert", yStock.SMAAlert , C.olNumber)

        #yOLI.UserProperties.Find("SMADate").Value=a_utils.DateTime2UTC(yStock.SMADate)
        yDT5=a_utils.DateTime2UTC(yStock.SMADate)
        SetOLIUsrProp(yOLI, "SMADate", yDT5 , C.olDateTime)

        #yOLI.UserProperties.Find("SMA").Value=yStock.SMA
        #print (" SMA=", yStock.SMA)
        SetOLIUsrProp(yOLI, "SMA", yStock.SMA , C.olCurrency)


        #yOLI.UserProperties.Find("Price").Value=yStock.Price
        SetOLIUsrProp(yOLI, "Price", yStock.Price , C.olCurrency)

        #yOLI.UserProperties.Find("PriceDate").Value=a_utils.DateTime2UTC(yStock.PriceDate)
        yDT6=a_utils.DateTime2UTC(yStock.PriceDate)
        print('yDT6=', yDT6)
        SetOLIUsrProp(yOLI, "PriceDate", yDT6, C.olDateTime)

        #yOLI.UserProperties.Find("Volume").Value=yStock.Volume
        SetOLIUsrProp(yOLI, "Volume", yStock.Volume , C.olNumber)

        SetOLIUsrProp(yOLI, "SMADays", yStock.SMADays , C.olNumber)

        yStock.Comment=yStock.Comment+'UpdateOLIFields() OK'  + ';'

    except:
        yStock.Comment=yStock.Comment+' UpdateOLIFields() not done '  + '; '

    

In [5]:
# u
# Recursive routine to get out of nested table in a word doc
# where is a range/selection within a table
# https://stackoverflow.com/questions/7226721/how-can-you-get-the-current-table-in-ms-word-vba
# cursor exist to outside of any table in MS Word
# n nest level 
def RngOutOfTables(yRng, n=0 ):
  
    if yRng.Information(C.wdWithInTable)==True and n < 500 :
        n=n+1 

        yRng.Collapse(Direction=C.wdCollapseStart) 
        yRow=yRng.Information(C.wdEndOfRangeRowNumber)
        yCell=yRng.Information(C.wdEndOfRangeColumnNumber)
        
        if yCell > yRng.Tables(1).Columns.Count: yCell=yRng.Tables(1).Columns.Count # a bug? 

     #   print (' inside table level = ', n)
     #   print (' At Row=', yRow,', Col=',yCell) 

       # print (' Table row=', yRng.Tables(1).Rows.Count,', Col=',yRng.Tables(1).Columns.Count)                  
        #print (' nested level= ', yRng.Tables(1).NestingLevel)
       
        #w yCellRng=yRng.Tables(1).Rows(yRow).Cells(yCell).Range
        yCellRng=yRng.Tables(1).Cell(Row=yRow, Column=yCell).Range
        yCellRng.MoveEnd(Unit=C.wdCharacter, Count=-1)  #remove the last cell ending char

        print ('Nested in', str(yRng.Tables(1).NestingLevel) ,"; cell(",yRow ,',', yCell,')=',  str( yCellRng.Text)  )
        #print ("cell(",yRow ,',', yCell,')=')
        yRng.Tables(1).Range.Collapse(Direction=C.wdCollapseStart)

        #yRng.Move(Unit=C.wdCell, Count=-1)  # NotW: cell, row, not working; Wrks: cell,word, character, sentence, wdParagraph,Story 
        #yRng.Move(Unit=C.wdCharacter, Count=+1) # to beyond table end     
        #yRng.Move(Unit=C.wdCharacter, Count=-1) #w to beyond table front will travse cell by cell

       ############################################### 

        mvUnit=C.wdParagraph

        m=yRng.Move(Unit=mvUnit, Count=-1)  # to the top of first level table if it is from inside of the table
        #w2 m=yRng.Move(Unit=C.wdStory, Count=-1)  # to the beginning of the whole doc
        if m==0: 
            m=yRng.Move(Unit=C.wdRow, Count=-1)
            if m==0:
                yRng.InsertBreak(Type=C.wdColumnBreak)
        #################################################


        yRng,n  =RngOutOfTables(yRng, n)
    

    return (yRng, n) 


In [6]:
import a_TDA_IF

def UpdateOLI(yOLI, yStock):

   UpdateOLIFields(yOLI, yStock )
  
   # get update the item 
   yInsp= yOLI.GetInspector
   yWDoc =yInsp.WordEditor
   #print('worddoc id=', id(yWDoc))
    
   if yWDoc.ProtectionType != C.wdNoProtection: 
       yWDoc.Unprotect()

   ySel=yWDoc.Windows(1).Selection  # get selection obj 

   #set range to the beginning for the doc
   yRng, n =RngOutOfTables(ySel.Range)
   #print('n=', n)

   #move to doc top
   yRng.Move(Unit=C.wdStory, Count=-1) #w, perfectly and one step!  

   #insert content
   yWDoc.Tables.Add(Range=ySel.Range, NumRows=3, NumColumns=1,
                    DefaultTableBehavior=C.wdWord9TableBehavior, AutoFitBehavior=C.wdAutoFitFixed)

   yWDoc.Tables(yWDoc.Tables.Count).Title = "Update"
   yWDoc.Tables(yWDoc.Tables.Count).ID = 123

   #first cell with time and date 
   yTString=datetime.datetime.now().strftime('%x')

   yRng =yWDoc.Characters(1)
   yRng.InsertBefore( yTString )

   print (type(ySel)) 
   #2nd cell with Stock.comment 
 
   yRng.Collapse(Direction=C.wdCollapseStart)
   yRng.Move(Unit=C.wdCell, Count=1 )  # when you have single column , this moves to the cell below 
   
   yRng.InsertBefore(yStock.Comment)
   #3nd cell for user notes:  
   yRng.Collapse(Direction=C.wdCollapseStart)
   yRng.Move(Unit=C.wdCell, Count=1 ) 

        

   yInsp.Close(C.olSave)  #works, must run to see result 
    #w message.Close(0)
    #w message.Save()
    
   

In [7]:
#test codes 
#04/17/21 official start point 
yWD= win32.gencache.EnsureDispatch("Word.Application")  # gencache.EnsureDispatch for wdConstant enumeration
yOL = win32.gencache.EnsureDispatch("Outlook.Application")  #w, needed for importing constants:
yNS = yOL.GetNamespace("MAPI")
yFolder = yNS.Folders['BXSelfCurrent'].Folders['BTHM'].Folders['0-outlook usage'].Folders['Test Run Outlook Usage'].Folders['Securities']

for yOLI in yFolder.Items:

    print('--------------------')
    yStock=InitStock (yOLI)
     
    

    yStock.GetHist() # include TA and saving 
    
    print('----')
    
   
    UpdateOLI(yOLI, yStock)
    print('--')
    
    
#yWD.Quit() 
#yOL.Quit() #w, but you want outlook to be on//



CRITICAL:root:get symbol market data, ok
CRITICAL:root:got quote update, <class 'pandas.core.frame.DataFrame'>(72, 7)
CRITICAL:root:UpdateTA()
CRITICAL:root:updated price=2297.76 volume=1130090 date=2021-04-15 22:13:20 SMA=2262.261999999999
CRITICAL:root:updated price=2297.76 volume=1130090SMAdate=2021-04-15 22:13:20 SMA=2262.261999999999
ERROR:root:DF2CSV() error in writing csv file
CRITICAL:root:SaveHist() path:  C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug\GOOG\GOOG_04_23_21.csv


--------------------
\r\n InitStock():; 
----
SetOLIUsrProp= SMAState , 1.0 , 3
SetOLIUserProp()  SMAState , 1.0 , accessor done
SetOLIUsrProp= SMAAlert , 0 , 3
SetOLIUserProp()  SMAAlert , 0 , accessor done
Nested in 1 ; cell( 1 , 1 )= 04/20/21
<class 'win32com.gen_py.00020905-0000-0000-C000-000000000046x0x8x5.Selection.Selection'>


CRITICAL:root:get symbol market data, ok
CRITICAL:root:got quote update, <class 'pandas.core.frame.DataFrame'>(72, 7)
CRITICAL:root:UpdateTA()
CRITICAL:root:updated price=2297.76 volume=1130090 date=2021-04-15 22:13:20 SMA=2262.261999999999
CRITICAL:root:updated price=2297.76 volume=1130090SMAdate=2021-04-15 22:13:20 SMA=2262.261999999999
ERROR:root:DF2CSV() error in writing csv file
CRITICAL:root:SaveHist() path:  C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug\FB\FB_04_23_21.csv


--
--------------------
\r\n InitStock():; 
----
SetOLIUsrProp= SMAState , 1.0 , 3
SetOLIUserProp()  SMAState , 1.0 , accessor done
SetOLIUsrProp= SMAAlert , 0 , 3
SetOLIUserProp()  SMAAlert , 0 , accessor done


CRITICAL:root:get symbol market data, ok
CRITICAL:root:got quote update, <class 'pandas.core.frame.DataFrame'>(72, 7)
CRITICAL:root:UpdateTA()
CRITICAL:root:updated price=2297.76 volume=1130090 date=2021-04-15 22:13:20 SMA=2262.261999999999
CRITICAL:root:updated price=2297.76 volume=1130090SMAdate=2021-04-15 22:13:20 SMA=2262.261999999999
ERROR:root:DF2CSV() error in writing csv file
CRITICAL:root:SaveHist() path:  C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug\AMZN\AMZN_04_23_21.csv


Nested in 1 ; cell( 1 , 1 )= 04/20/21
<class 'win32com.gen_py.00020905-0000-0000-C000-000000000046x0x8x5.Selection.Selection'>
--
--------------------
\r\n InitStock():; 
----
SetOLIUsrProp= SMAState , 1.0 , 3
SetOLIUserProp()  SMAState , 1.0 , accessor done
SetOLIUsrProp= SMAAlert , 0 , 3
SetOLIUserProp()  SMAAlert , 0 , accessor done
Nested in 1 ; cell( 1 , 1 )= 04/20/21
<class 'win32com.gen_py.00020905-0000-0000-C000-000000000046x0x8x5.Selection.Selection'>
--


In [ ]:
# original, deprecated 041721
##import win32com.client as win32
#https://docs.microsoft.com/en-us/previous-versions/office/developer/office-2010/cc313152(v=office.12)
#https://docs.microsoft.com/en-us/previous-versions/office/developer/office-2010/ee861520(v=office.14)


#ok:   C = win32com.client.constants
# https://stackoverflow.com/questions/21465810/accessing-enumeration-constants-in-excel-com-using-python-and-win32com
##from win32com.client import constants as C


#nw outlook = win32.Dispatch("Outlook.Application") #nw for importing VBA constants, needs gencache.EnsureDispatch
##yWord= win32.gencache.EnsureDispatch("Word.Application")  # for wdConstant enumeration
yWord1= win32.gencache.EnsureDispatch("Word.Application")  # for wdConstant enumeration
outlook1 = win32.gencache.EnsureDispatch("Outlook.Application")  #w, needed for importing constants:
mapi1 = outlook1.GetNamespace("MAPI")

# prove that C is defined by import: w: print (C.olAlignCenter)
print (C.olAlignCenter, C.wdNoProtection)
your_folder = mapi1.Folders['BXSelfCurrent'].Folders['BTHM'].Folders['0-outlook usage'].Folders['Test Run Outlook Usage'].Folders['Securities']


for message in your_folder.Items:
    ##print(message.UserProperties.Find("Sec").Value,  message.UserProperties.Find("price").Value, message.Subject) 
    # if not a keyword and not empty
    message.UserProperties.Find("price").Value=88.88
    ##print(message.UserProperties.Find("Sec").Value,  message.UserProperties.Find("price").Value, message.Subject)

    UpdateOLI(message)  ##testing 


1 -1
symbol=  GOOG
 get symbol market data
Nested in 1 ; cell( 1 , 1 )= 04/18/21 21:54:36: 
symbol=  AMD
 get symbol market data
Nested in 1 ; cell( 1 , 1 )= 04/18/21 21:54:36: 


In [ ]:
##########testing oli datetime field
import win32com.client as win32
from win32com.client import constants as C
import datetime 
import time

yWord1= win32.gencache.EnsureDispatch("Word.Application")  # for wdConstant enumeration
outlook1 = win32.gencache.EnsureDispatch("Outlook.Application")  #w, needed for importing constants:
mapi1 = outlook1.GetNamespace("MAPI")


yOLI1=outlook1.ActiveExplorer().Selection.Item(1)
message=yOLI1

print(message.UserProperties.Find("Price").Value)


yOLDT=message.UserProperties.Find("PriceDate").Value
print(type(yOLDT), yOLDT)

ySTM=1262332800
yDT1=datetime.datetime.fromtimestamp(ySTM)

yPYTM=datetime.datetime.now() 
yPYTM=yDT1
print("pyDT=", yPYTM )

yPYTM1= yPYTM-datetime.timedelta(hours=8 )
message.UserProperties.Find("PriceDate").Value= yPYTM1 
yOLDT=message.UserProperties.Find("PriceDate").Value
print(type(yOLDT), yOLDT)


yOLVlm=yOLDT=message.UserProperties.Find("Volume")
print(type(yOLVlm))





888.88
<class 'pywintypes.datetime'> 2010-01-01 00:00:00+00:00
pyDT= 2010-01-01 00:00:00
<class 'pywintypes.datetime'> 2010-01-01 00:00:00+00:00
<class 'win32com.gen_py.00062FFF-0000-0000-C000-000000000046x0x9x4.UserProperty.UserProperty'>


In [ ]:
#############################################
# quick test on active document 
###########################################
yAWdEd=outlook.ActiveInspector().WordEditor
yASel=yAWdEd.Windows(1).Selection 



#yARng1, n =RngOutOfTables(yASel.Range)
#print('level=', n)

#yASel.Range.Collapse()
#yARng1.Move(Unit=C.wdStory, Count=-1) #w, perfectly and one step!  
#yARng1.Select()



In [ ]:
# deprecated 041721
import datetime 
import time
try: 
    # set word editor
    # https://stackoverflow.com/questions/23824341/outlook-vba-replace-inline-object-with-text
    print(message.Class)
    yInsp= message.GetInspector
    print ("Inspector", yInsp.Class, yInsp.EditorType)  # olEditWord =4 
except:
    
    ##################################################
    # only if item is allowed by Winodows to be in foreground
    #
    #yInsp.Display
    #yInsp.Activate
    #yInsp.CommandBars.ExecuteMso ("EditMessage")  # works when inspector is open
    ##############################################

    #Wordedit() returns a document object ! 
    yWordEd =yInsp.WordEditor
    if yWordEd.ProtectionType != C.wdNoProtection: yWordEd.Unprotect()
    print('worddoc obj id',id(yWordEd))
    print ( 'word doc name', yWordEd.Name)


    # for your understanding only,
    # this is not need since you start from list of OL items, while OL VBA insert table starts from active inspector  
    #  yActDoc=yWordEd.Application.ActiveDocument   # no active document (forefront, mouse/kb focused)  at this step 
    #  print(id(yActDoc))
    #  print(yActDoc.Name)
    #  yActDoc.Unprotect()


    yTM=datetime.datetime.now()

    yTString= yTM.strftime('%x')+ " "+ yTM.strftime('%X') +'; ' + r'\r\n ' 
    print(yTString)
    #yActDoc.ActiveWindow.View.ReadingLayout = False

    #https://stackoverflow.com/questions/5453998/python-instance-to-com-object
    #https://www.pythonstudio.us/introduction-2/implementing-com-objects-in-python.html
    #yActDoc.Tables.Add(Range=yRange, NumRows=1, NumColumns=1) 
        
    #yActDoc.Tables(.Tables.Count).Title = "testing"
    #yActDoc.Tables(.Tables.Count).ID = .Tables.Count

        ySel=yWordEd.Windows(1).Selection  # start of the first cell 

        #ySel.Collapse()
        #ySel.Move(Unit=C.wdWord, Count=-1)

        yARng1, n =RngOutOfTables(ySel.Range)

        #nn ySel.Move(C.wdStory,-1) 
        #nn ySel.HomeKey(Unit=C.wdStory)
        yWordEd.Tables.Add(Range=yARng1, NumRows=1, NumColumns=1,
        DefaultTableBehavior=C.wdWord9TableBehavior, AutoFitBehavior=C.wdAutoFitFixed)

        yWordEd.Tables(yWordEd.Tables.Count).Title = "testing"
        yWordEd.Tables(yWordEd.Tables.Count).ID = yWordEd.Tables.Count

        #
        # find table at the cursor position
        # https://stackoverflow.com/questions/7226721/how-can-you-get-the-current-table-in-ms-word-vba
        #
        #yRng1, n =RngOutOfTables(ySel.Range)
        #ySel=yRng1.Select

        yTString=yTString + "; table count=" + str(n)


        ySel =yWordEd.Characters(1)
        ySel.InsertBefore( yTString )

        #if yWordEd.Saved != True: yWordEd.Save() #nw: prompt you the save a word doc
        print (C.olSave)
        print (yInsp.CurrentItem.Subject)
        # message.Save()

        yInsp.Close(C.olSave)  #works, must run to see result 
        #w message.Close(0)
        #w message.Save()

        #yWord.Quit() 
        #w, but you want outlook to be on// outlook.Quit()
